# General Things

In [ ]:
## CREATE

# to make a directory
mkdir directory

# to make multiple directories at the same time
mkdir directory1 directory2 directory3

# to write a file
nano filename

# the best partition to use is bmh, 
# On the high2 partition: 256 CPUs and 500GB of RAM
# On the bmh partition: 128 CPUs and 1TB of RAM

----------------------------

## CHANGE


# to move files up a folder
mv * ../

# to move directory into another directory (that already exists)
mv directorytomove directorythatexists

# to "rename" a file (for example, rename something called "error" to "err")
mv error err

# to copy a file and rename (if in same folder)
cp file ./filenew

cp qual_filter.sh ./qual_filter2.sh

# to copy a file and move to different directory
cp file directory/
cp file1 file2 file3 directory/

----------------------------

## MONITOR

# to view a file
less filename

#to access personal folders (on your own windows computer)
cd /mnt/c/

# to see status of jobs (input username, mine is seuge)
squeue -u seuge

# to cancel jobs (input username, mine is seuge)
scancel -u seuge

# view size of files in human readable form
ls -lah
ls

# counting lines in a file
wc -l filename

# list one file per line    
ls -1

# counting number of files in a directory
ls -1 | wc -l

# to see the beginning of many files in a directory
head -5 *.txt
# with 5 being the number lines, you can edit this with how many lines you want to see

# to search for something in files
grep -r 'path' -e 'something'
grep -r 'home/seuge91/fire/reads/log' -e '53927615'

# to count number of lines in a zipped file
zcat 'file name' | wc -l

# to view the next file in less, press “:” and then hit “n”

# to list installed packages and their versions (either command seems to work just fine)
pip list
conda list

----------------------------

# DELETE

# to remove a file 
rm filename

# to remove all files in a folder
rm *

# to remove a directory by overriding computer
rm -r nameofdirectory

----------------------------

# STOP/EXIT

# exit nano by ctrl-x, Y (to save), and enter

# exit less
q

# stop a command line execution
CTRL+c

## Download Data

These will be fastq.gz files, paired-end reads

Downloading data from genome center https://dnatech.genomecenter.ucdavis.edu/archiving-slims-data/ (rsync)

In [ ]:
# screen to have something running while doing something else (like making copies)
screen 

# move into directory you want the data to go (don't forget the dot at the end!!!)
rsync -avL slimsdata.genomecenter.ucdavis.edu::slims/5ipm20rfhg . 

rsync -avL slimsdata.genomecenter.ucdavis.edu::slims/dg4d3xq5bn . 

# folder and data should start transferring immediately

# detach from screen: ctrl a + d

# to see screens running
screen -r

# to end screen
exit

## Move Data Around

In [ ]:
mv * /home/seuge91/fire/

mv * /home/seuge91/fire/reads/raw3

# Anatomy of a Project Directory

In [ ]:
bowtie
    ref
drep
    all
        all_dRep
            data
            data_tables
            dereplicated_genomes 
            figures
            log
        split_contigs
    drep.sh
    vibrant_good
megahit
    err
    log
    contigs
    renamed_contigs
reads
    err
    log
    raw
    rmphix
    rmphix_unpaired
    stats
    trimmed
    unpaired
sampleIDs.txt
    # to create this
    cd /home/seuge91/fire/reads/raw
    ls -1 | cut -f1,2,3 -d_ | sort | uniq > sampleIDs.txt
    # takes the first three elements of the sample name that are separated by the "_" delimitator
    # remove extraneous labels like @md5Sum.md5
    # make a txt file for each batch of raw reads (aka, sampleIDs2.txt, sampleIDs3.txt)
unfinishedIDs.txt
scripts
    qual_filter.sh
trimmed
vibrant
    err
    log
    # all the _vibrant directories for each sample

# Quality Filtering

In [ ]:
nano qual_filter.sh

#!/bin/bash
#SBATCH -D /home/seuge91/fire/scripts/
#SBATCH --job-name=qual_filter
#SBATCH --nodes=1
#SBATCH --time=04:00:00
#SBATCH --ntasks=8
#SBATCH --mem=32GB
#SBATCH --partition=bmm

# for calculating the amount of time the job takes
begin=`date +%s`
echo $HOSTNAME

# loading modules
module load bbmap
source /home/csantosm/initconda
conda activate TRIMMOMATIC

# running commands
path=${1}
sample=${2}

cd ${path}

trimmomatic PE -threads 8 -phred33 \
  ./reads/raw/${sample}_R1_001.fq.gz ./reads/raw/${sample}_R2_001.fq.gz \
  ./reads/trimmed/${sample}_R1_trimmed.fq.gz ./reads/unpaired/${sample}_R1_unpaired.fq.gz \
  ./reads/trimmed/${sample}_R2_trimmed.fq.gz ./reads/unpaired/${sample}_R2_unpaired.fq.gz \
  ILLUMINACLIP:/home/csantosm/databases/TruSeq3-PE.fa:2:30:10 \
  SLIDINGWINDOW:4:30 MINLEN:50

  bbduk.sh \
    in1=./reads/trimmed/${sample}_R1_trimmed.fq.gz \
    in2=./reads/trimmed/${sample}_R2_trimmed.fq.gz \
    out1=./reads/rmphix/${sample}_R1_rmphix.fq.gz \
    out2=./reads/rmphix/${sample}_R2_rmphix.fq.gz \
    ref=/home/csantosm/databases/phix174_ill.ref.fa \
    k=31 \
    hdist=1 \
    stats=./reads/stats/${sample}_stats.txt \
    -Xmx20g

  bbduk.sh \
    in=./reads/unpaired/${sample}_R1_unpaired.fq.gz \
    out=./reads/rmphix_unpaired/${sample}_R1_rmphix_unpaired.fq.gz \
    ref=/home/csantosm/databases/phix174_ill.ref.fa \
    k=31 \
    hdist=1 \
    stats=./reads/stats/${sample}_R1_stats.txt \
    -Xmx20g

  bbduk.sh \
    in=./reads/unpaired/${sample}_R2_unpaired.fq.gz \
    out=./reads/rmphix_unpaired/${sample}_R2_rmphix_unpaired.fq.gz \
    ref=/home/csantosm/databases/phix174_ill.ref.fa \
    k=31 \
    hdist=1 \
    stats=./reads/stats/${sample}_R2_stats.txt \
    -Xmx20g

# finished commands

# getting end time to calculate time elapsed
end=`date +%s`
elapsed=`expr $end - $begin`
echo Time taken: $elapsed
    
----------------------------

# to run commands (batch 1)

bash
for sample in $(</home/seuge91/fire/sampleIDs.txt)
do
sbatch \
--output=/home/seuge91/fire/reads/log/${sample}.qf.log \
--error=/home/seuge91/fire/reads/err/${sample}.qf.err \
/home/seuge91/fire/scripts/qual_filter.sh /home/seuge91/fire/ $sample
done

# to run commands (batch 2)

bash
for sample in $(</home/seuge91/fire/sampleIDs2.txt)
do
sbatch \
--output=/home/seuge91/fire/reads/log/${sample}.qf.log \
--error=/home/seuge91/fire/reads/err/${sample}.qf.err \
/home/seuge91/fire/scripts/qual_filter2.sh /home/seuge91/fire/ $sample
done

# to run commands (batch 3)

bash
for sample in $(</home/seuge91/fire/sampleIDs3.txt)
do
sbatch \
--output=/home/seuge91/fire/reads/log/${sample}.qf.log \
--error=/home/seuge91/fire/reads/err/${sample}.qf.err \
/home/seuge91/fire/scripts/qual_filter3.sh /home/seuge91/fire/ $sample
done

# to run commands (heat)

bash
for sample in $(</home/seuge91/heat/sampleIDs.txt)
do
sbatch \
--output=/home/seuge91/heat/reads/log/${sample}.qf.log \
--error=/home/seuge91/heat/reads/err/${sample}.qf.err \
/home/seuge91/heat/scripts/qual_filter.sh /home/seuge91/heat/ $sample
done

# Concatenate files (if multiple batches!)

In [ ]:
## create sample ID text file that only has the first part of the sample ID 
## that is consistent among all batches that you can use to concatenate files

cd /home/seuge91/fire/reads/raw
    ls -1 | cut -f1 -d_ | sort | uniq > sample.txt

# for paired reads

nano catpaired.sh

----------------------------

#!/bin/bash
#SBATCH -D /home/seuge91/fire/reads/
#SBATCH --job-name=catpaired
#SBATCH --nodes=1
#SBATCH -t 8:00:00
#SBATCH --ntasks=4
#SBATCH --partition=bmm

for file in $(</home/seuge91/fire/sample.txt)
do
cat rmphix/${file}_*_*_R1_*.fq.gz rmphix2/${file}_*_*_R1_*.fq.gz rmphix3/${file}_*_*_R1_*.fq.gz >> ${file}_R1_rmphix_combined.fq.gz
cat rmphix/${file}_*_*_R2_*.fq.gz rmphix2/${file}_*_*_R2_*.fq.gz rmphix3/${file}_*_*_R2_*.fq.gz >> ${file}_R2_rmphix_combined.fq.gz
done

----------------------------

# to run command

sbatch catpaired.sh

# for unpaired reads

nano catunpaired.sh

----------------------------

#!/bin/bash
#SBATCH -D /home/seuge91/fire/reads/
#SBATCH --job-name=catunpaired
#SBATCH --nodes=1
#SBATCH -t 8:00:00
#SBATCH --ntasks=4
#SBATCH --partition=bmm

for file in $(</home/seuge91/fire/sample.txt)
do
cat rmphix_unpaired/${file}_*_*_R1_*.fq.gz rmphix_unpaired2/${file}_*_*_R1_*.fq.gz rmphix_unpaired3/${file}_*_*_R1_*.fq.gz >> ${file}_R1_rmphix_combined_unpaired.fq.gz
cat rmphix_unpaired/${file}_*_*_R2_*.fq.gz rmphix_unpaired2/${file}_*_*_R2_*.fq.gz rmphix_unpaired3/${file}_*_*_R2_*.fq.gz >> ${file}_R2_rmphix_combined_unpaired.fq.gz
done

----------------------------

# to run command

sbatch catunpaired.sh

# Assembly

## megahit

progressive k-mer assembly, starts with k-mer size of 27, and then goes up by k-mer step of 10
our settings starts k-mer at 27 and end at 87 (--presets meta-large)
builds graph with k-mer 27 first, those are the first contigs
then uses those contigs in addition to all the reads to build graph with k-mer 37
small differences will "explode the graph" as k-mer size increases


In [ ]:



cd ~/fire/scripts
nano megahit.sh

----------------------------

#!/bin/bash
#SBATCH -D /home/seuge91/fire/scripts/
#SBATCH --job-name=megahit
#SBATCH --nodes=1
#SBATCH -t 24:00:00
#SBATCH --ntasks=8
#SBATCH --partition=bmh

# for calculaticdng the amount of time the job takes
begin=`date +%s`
echo $HOSTNAME

# loading modules
source /home/csantosm/initconda
conda activate MEGAHIT

# move to rawreads folder
cd /home/seuge91/fire/reads/
sample=${1}

megahit -1 ./rmphix_combined/${sample}_R1_rmphix_combined.fq.gz \
-2 ./rmphix_combined/${sample}_R2_rmphix_combined.fq.gz \
-r ./rmphix_combined_unpaired/${sample}_R1_rmphix_combined_unpaired.fq.gz,./rmphix_combined_unpaired/${sample}_R2_rmphix_combined_unpaired.fq.gz \
-o ../megahit/${sample} \
--out-prefix ${sample} \
--min-contig-len 10000 --presets meta-large \
-t 8 --continue

# getting end time to calculate time elapsed
end=`date +%s`
elapsed=`expr $end - $begin`
echo Time taken: $elapsed
    
----------------------------

# to run commands

bash
for sample in $(</home/seuge91/fire/sample.txt)
do
sbatch \
--output=/home/seuge91/fire/megahit/log/${sample}.mh.log \
--error=/home/seuge91/fire/megahit/err/${sample}.mh.err \
/home/seuge91/fire/scripts/megahit.sh $sample
done

bash
for sample in $(</home/seuge91/heat/sampleIDs.txt)
do
sbatch \
--output=/home/seuge91/heat/megahit/log/${sample}.mh.log \
--error=/home/seuge91/heat/megahit/err/${sample}.mh.err \
/home/seuge91/heat/scripts/megahit.sh $sample
done

___

bash
for sample in $(</home/seuge91/fire/newsample.txt)
do
sbatch \
--output=/home/seuge91/fire/megahit/log/${sample}.mh.log \
--error=/home/seuge91/fire/megahit/err/${sample}.mh.err \
/home/seuge91/fire/scripts/newmegahit.sh $sample
done

bash
for sample in $(</home/seuge91/fire/unfinished2.txt)
do
sbatch \
--output=/home/seuge91/fire/megahit/log/${sample}.mh.log \
--error=/home/seuge91/fire/megahit/err/${sample}.mh.err \
/home/seuge91/fire/scripts/megahit.sh $sample
done

bash
for sample in $(</home/seuge91/fire/unfinished3.txt)
do
sbatch \
--output=/home/seuge91/fire/megahit/log/${sample}.mh.log \
--error=/home/seuge91/fire/megahit/err/${sample}.mh.err \
/home/seuge91/fire/scripts/newmegahit.sh $sample
done

bash
for sample in $(</home/seuge91/fire/unfinished4.txt)
do
sbatch \
--output=/home/seuge91/fire/megahit/log/${sample}.mh.log \
--error=/home/seuge91/fire/megahit/err/${sample}.mh.err \
/home/seuge91/fire/scripts/megahit.sh $sample
done

# to run samples that were suspended/unfinished
# make updated sampleIDs list with samples that need to be resubmitted 
# I made unfinishedIDs.txt 

nano unfinishedIDs.txt

# to run commands

bash
for sample in $(</home/seuge91/heat/unfinishedIDs.txt)
do
sbatch \
--output=/home/seuge91/heat/megahit/log/${sample}.mh.log \
--error=/home/seuge91/heat/megahit/err/${sample}.mh.err \
/home/seuge91/heat/scripts/megahit.sh $sample
done

----------------------------

# to count how many contigs assembled in each sample

grep -rc "k127" 


# Viral Detection

In [ ]:
cd megahit
mkdir contigs
mkdir renamed_contigs

# move all contig fasta files into one folder
cd /home/seuge91/fire/megahit
mv */*contigs.fa contigs
cd contigs

# rename contig files so that it shows exactly what sample each contig came from
module load bbmap

for sample in $(<../../sample.txt)
do
rename.sh in=${sample}.contigs.fa out=${sample}.renamed.contigs.fa prefix=${sample}_contig_
done

for sample in $(<../../sampleIDs.txt)
do
rename.sh in=${sample}.contigs.fa out=${sample}.renamed.contigs.fa prefix=${sample}_contig_
done

# I accidentally deleted the contents of C10_S58_L004.renamed.contigs.fa - I tried running the below script but it does not work
rename.sh in=C10_S58_L004.contigs.fa out=C10_S58_L004.renamed.contigs.fa prefix=C10_S58_L004_contig_

# it didn't work because java wasnt installed
# had to load deprecated version for it to work
module load deprecated/java



mv *renamed.contigs.fa ../renamed_contigs

cd /home/seuge91/fire
mkdir vibrant
cd vibrant
mkdir err log

# make vibrant script
nano vibrant.sh

#below is an old script (keep scrolling for most updated one)
----------------------------

#!/bin/bash
#SBATCH -D /home/seuge91/fire/scripts/
#SBATCH --job-name=vbrnt
#SBATCH --nodes=1
#SBATCH -t 12:00:00
#SBATCH --ntasks=4
#SBATCH --partition=bmh

# for calculating the amount of time the job takes
begin=`date +%s`
echo $HOSTNAME

#activate personal conda env
source /home/csantosm/initconda
conda activate VIBRANT

# move to main folder
cd /home/seuge91/fire/
sample=${1}

VIBRANT_run.py -i ./megahit/renamed_contigs/${sample}.renamed.contigs.fa \
-folder ./vibrant/${sample}_vibrant \
-t 4 -f nucl -virome

#getting end time to calculate time elapsed
end=`date +%s`
elapsed=`expr $end - $begin`
echo Time taken: $elapsed
   
----------------------------
#below is an old script (keep scrolling for most updated one)
----------------------------

#!/bin/bash
#SBATCH -D /home/seuge91/heat/scripts/
#SBATCH --job-name=vbrnt
#SBATCH --nodes=1
#SBATCH -t 12:00:00
#SBATCH --ntasks=4
#SBATCH --partition=bmh

# for calculating the amount of time the job takes
begin=`date +%s`
echo $HOSTNAME

#activate personal conda env
source /home/csantosm/initconda
conda activate VIBRANT

# move to main folder
cd /home/seuge91/heat/
sample=${1}

VIBRANT_run.py -i ./megahit/renamed_contigs/${sample}.renamed.contigs.fa \
-folder ./vibrant/${sample}_vibrant \
-t 4 -f nucl -virome

#getting end time to calculate time elapsed
end=`date +%s`
elapsed=`expr $end - $begin`
echo Time taken: $elapsed
    
----------------------------

# for some reason these files were incomplete, I'm going to try running again with original script

###new script 8/15/2023 (did not work, a bunch of temp files???)

mkdir vibrant_sara
cd vibrant_sara
mkdir err log
----------------------------

#!/bin/bash
#SBATCH -D /home/seuge91/heat/scripts/
#SBATCH --job-name=vbrnt
#SBATCH --nodes=1
#SBATCH -t 12:00:00
#SBATCH --ntasks=4
#SBATCH --partition=bmh

# for calculating the amount of time the job takes
begin=`date +%s`
echo $HOSTNAME

#activate personal conda env
source /home/csantosm/initconda
conda activate vibrant_sara

# move to main folder
cd /home/seuge91/heat/
sample=${1}

~/VIBRANT/VIBRANT/VIBRANT_run.py -i ./megahit/renamed_contigs/${sample}.renamed.contigs.fa \
-folder ./vibrant_sara/${sample}_vibrant \
-t 4 -f nucl -virome

#getting end time to calculate time elapsed
end=`date +%s`
elapsed=`expr $end - $begin`
echo Time taken: $elapsed
    
----------------------------

# to run commands

bash
for sample in $(</home/seuge91/heat/sampleIDs.txt)
do
sbatch \
--output=/home/seuge91/heat/vibrant_sara/log/${sample}.vb.log \
--error=/home/seuge91/heat/vibrant_sara/err/${sample}.vb.err \
/home/seuge91/heat/scripts/vibrant_sara.sh $sample
done

# to run commands on suspended jobs (make sure to edit unfinishedIDs.txt)

bash
for sample in $(</home/seuge91/fire/unfinishedIDs.txt)
do
sbatch \
--output=/home/seuge91/fire/vibrant/log/${sample}.vb.log \
--error=/home/seuge91/fire/vibrant/err/${sample}.vb.err \
/home/seuge91/fire/scripts/vibrant.sh $sample
done



# redo script:

----------------------------

#!/bin/bash
#SBATCH -D /home/seuge91/heat/scripts/
#SBATCH --job-name=vbrnt
#SBATCH --nodes=1
#SBATCH -t 12:00:00
#SBATCH --ntasks=4
#SBATCH --partition=bmh

# for calculating the amount of time the job takes
begin=`date +%s`
echo $HOSTNAME

#activate personal conda env
source /home/csantosm/initconda
conda activate VIBRANT

# move to main folder
cd /home/seuge91/heat/
sample=${1}

VIBRANT_run.py -i ./megahit/renamed_contigs/${sample}.renamed.contigs.fa \
-folder ./vibrant_redo/${sample}_vibrant \
-t 4 -f nucl -virome

#getting end time to calculate time elapsed
end=`date +%s`
elapsed=`expr $end - $begin`
echo Time taken: $elapsed
   
----------------------------

# to run commands on jobs to redo

bash
for sample in $(</home/seuge91/heat/redoIDs.txt)
do
sbatch \
--output=/home/seuge91/heat/vibrant_redo/log/${sample}.vb.log \
--error=/home/seuge91/heat/vibrant_redo/err/${sample}.vb.err \
/home/seuge91/heat/scripts/vibrant.sh $sample
done



In [ ]:
# count number of phages in each sample (if samples were not renamed)

grep -rc "k87" # but this gives you a number for everything

grep -rc "k87" --include="*phages_combined.txt" # this specifies the filename to look in

# count number of phages in each sample (if samples were renamed)

grep -rc "contig" --include="*phages_combined.txt"

# count number of circular, high, and medium quality drafts in each sample
grep -rc 'circular\|high\|medium' --include="*genome_quality*"

grep -rc 'circular' --include="*genome_quality*"

# to separate output into fields for easy table export
cd /home/seuge91/heat/vibrant
grep -rc 'low' --include="*genome_quality*" | awk -F[:/] '{print $1,$5}' > lowcontigs.txt
grep -rc 'medium' --include="*genome_quality*" | awk -F[:/] '{print $1,$5}' > mediumcontigs.txt
grep -rc 'high' --include="*genome_quality*" | awk -F[:/] '{print $1,$5}' > highcontigs.txt
grep -rc 'circular' --include="*genome_quality*" | awk -F[:/] '{print $1,$5}' > circularcontigs.txt

grep -rc "contig" --include="*phages_combined.fna" | awk -F[:] '{print $1,$2}' > ../viralcontigs.txt

#to download these files
rsync -r --progress seuge91@farm:~/heat/vibrant/*contigs.txt /mnt/c/Users/segeo/Downloads
rsync -r --progress seuge91@farm:~/heat/vibrant/viralcontigs.txt /mnt/c/Users/segeo/Downloads

In [ ]:
# downloading files (you need to be signed out)

# download files of each sample into appropriate folder on box

rsync -r --progress seuge91@farm:~/heat/vibrant/*_vibrant/*renamed.contigs/VIBRANT_figures_* /mnt/c/Users/segeo/Downloads
    
rsync -r --progress seuge91@farm:~/blodgett/Project_JESG_Nova348P_Geonczy/vibrant/space.txt /mnt/c/Users/segeo/Downloads

rsync -r --progress seuge91@farm:~/heat/vibrant/*contigs.txt /mnt/c/Users/segeo/Downloads
 

# Dereplication
## https://drep.readthedocs.io/en/latest/overview.html
https://drep.readthedocs.io/en/latest/module_descriptions.html

In [ ]:
# cleaned-up! (this is for heat study)

# prepare files for dereplication
# in vibrant folder, make new directory for vibrant contigs
mkdir updated_vibrant_contigs

# copy all contigs to directory
cp */*/VIBRANT_phages*/*phages_combined.fna ./vibrant_contigs

# in project directory, make drep directory
mkdir updated_drep

# make subdirectory
mkdir all

# make folder for split contigs in each subdirectory
mkdir split_contigs

# move all viral contigs from each sample from vibrant to the folder called "split_contigs" ...
# ...by first moving" into vibrant folder and then vibrant_contigs directory
cd vibrant
cd updated_vibrant_contigs

# concatenate all viral contigs into a single file
cat *phages_combined.fna > all.vib.contigs.fna

# count how many total contigs
grep -c contig all.vib.contigs.fna

# first cluster using CD HIT
# rename file to use for CD Hit (input.fna)
cp all.vib.contigs.fna ./input.fna

# create CD HIT script
nano cdhit.sh

-----------------
#!/bin/bash
#SBATCH --job-name=cdhit
#SBATCH --nodes=1
#SBATCH -t 24:00:00
#SBATCH --ntasks=10
#SBATCH --output=cdhit%j.out
#SBATCH --error=cdhit%j.err
#SBATCH --partition=bmh

module load cdhit
cd-hit-est -i input.fna -o clustered_heat.fna \
-c 0.95 -aS 0.85 -M 7000 -T 10
-----------------

# run CD HIT script
sbatch cdhit.sh

# number of contigs after CD HIT: 141,638 --> 34,948

# now run dereplication

# take the concatenated file and move it to the appropriate split_contig folder within a drep/"subset"/split_contigs
mv clustered_heat.fna cd /home/seuge91/heat/updated_drep/all/split_contigs

# this works even if there is some weird error that shows up

# separate the concatenated fasta file into multiple ones
# use screen if you have a lot of contigs (this will take awhile)
awk '/^>/ {OUT=substr($0,2) ".fa"}; OUT {print >OUT}' *.fna 

    # if using screen...you can monitor how close it is to done by counting file names
        ls -1 | wc -l

# once done, remove the concatenated file from folder (you can move up a folder)
mv clustered_heat.fna ../

# make updated_drep script
nano updated_drep.sh
-----------------------------------------------------------------------------------------

#!/bin/bash
#SBATCH -D /home/seuge91/heat/updated_drep
#SBATCH --job-name=drep
#SBATCH --nodes=1
#SBATCH -t 72:00:00
#SBATCH --ntasks=24
#SBATCH --mem=900GB
#SBATCH --partition=bmh

# for calculating the amount of time the job takes
begin=`date +%s`
echo $HOSTNAME

# activate personal conda env
source /home/csantosm/initconda
conda activate DREP

treatment=${1}

cd /home/seuge91/heat/updated_drep/${treatment}

dRep dereplicate ./${treatment}_dRep \
-g ./split_contigs/*.fa \
--S_algorithm ANImf \
-sa 0.95 \
-nc 0.85 \
-l 10000 \
-N50W 0 \
-sizeW 1 \
--ignoreGenomeQuality \
--clusterAlg single \
-p 24

#getting end time to calculate time elapsed
end=`date +%s`
elapsed=`expr $end - $begin`
echo Time taken: $elapsed
    
-----------------------------------------------------------------------------------------

# run the script giving the variable of the subset folder that has the split_contigs directory
sbatch updated_drep.sh all

-----------------------------------------------------------------------------------------

# number of contigs after drep: 34,948 --> 18,869

# to zip all.drep.contigs.fa file and download so that database of dereplicated vOTUs can be uploaded to zenodo
gzip -k all.drep.contigs.fa
scp seuge91@farm.cse.ucdavis.edu:/home/seuge91/heat/updated_bowtie/all.drep.contigs.fa.gz /mnt/c/Users/segeo/Downloads



In [ ]:
# this cell is messy so refer to above cell /\

# prepare files for dereplication
# in vibrant folder, make new directory for vibrant contigs
mkdir vibrant_contigs

# copy all contigs to directory
cp */*/VIBRANT_phages*/*phages_combined.fna ./vibrant_contigs

# in project directory, make drep directory
mkdir drep

# make subdirectory
mkdir all

# make folder for split contigs in each subdirectory
mkdir split_contigs

# move all viral contigs from each sample from vibrant to the folder called "split_contigs" ...
# ...by first moving" into vibrant folder and then vibrant_contigs directory
cd vibrant
cd vibrant_contigs

# concatenate all viral contigs into a single file
cat *phages_combined.fna > all.vib.contigs.fna

# count how many total contigs
grep -c contig all.vib.contigs.fna

    # fire data - ~115,000, maybe too much?
    # updated fire data - 404,439
    # IF THIS IS A LOT MORE THAN 100,000 YOU SHOULD ONLY TAKE MEDIUM AND HIGH QUALITY CONTIGS
    # heat data = 139,398 contigs

# take the concatenated file and move it to the appropriate split_contig folder within a drep/"subset"/split_contigs
mv all.vib.contigs.fna cd /home/seuge91/fire/drep/all/split_contigs
mv all.vib.contigs.fna cd /home/seuge91/heat/drep/all/split_contigs

# this works even if there is some weird error that shows up

# separate the concatenated fasta file into multiple ones
awk '/^>/ {OUT=substr($0,2) ".fa"}; OUT {print >OUT}' *.fna 

    #is this supposed to take a long time? yes, if you have a lot of contigs...use screen
    # you can monitor how close it is to done by counting file names
        ls -1 | wc -l

# once done, remove the concatenated file
rm all.vib.contigs.fna

-----------------------------------------------------------------------------------------

#!/bin/bash
#SBATCH -D /home/seuge91/fire/drep
#SBATCH --job-name=drep
#SBATCH --nodes=1
#SBATCH -t 72:00:00
#SBATCH --ntasks=24
#SBATCH --mem=900GB
#SBATCH --partition=bmh

# for calculating the amount of time the job takes
begin=`date +%s`
echo $HOSTNAME

# activate personal conda env
source /home/csantosm/initconda
conda activate DREP

treatment=${1}

cd /home/seuge91/fire/drep/${treatment}

dRep dereplicate ./${treatment}_dRep \
-g ./split_contigs/*.fa \
--S_algorithm ANImf \
-sa 0.95 \
-nc 0.85 \
-l 10000 \
-N50W 0 \
-sizeW 1 \
--ignoreGenomeQuality \
--clusterAlg single \
-p 24

#getting end time to calculate time elapsed
end=`date +%s`
elapsed=`expr $end - $begin`
echo Time taken: $elapsed
    
-----------------------------------------------------------------------------------------

# run the script giving the variable of the subset folder that has the split_contigs directory
sbatch drep.sh all

-----------------------------------------------------------------------------------------

# may run out of memory, check slurm output files in drep directory

# if this is the case, just use subset of medium and high quality genome drafts
cat vibrant_contigs/*phages_combined.fna > all.vib.contigs.fna

# to see how many contigs in all.vib.contigs.fna
grep '>' all.vib.contigs.fna | wc -l

# to get list of contigs that are medium and high quality
grep 'circular\|high\|medium' */*/VIBRANT_results*/VIBRANT_genome_quality_* | cut -f1 | cut -f2 -d: | sort | uniq > good.vib.ids

# to see how many contigs in file
wc -l good.vib.ids

    # fire - 107,053
    
# load Chris's conda space
source /home/csantosm/initconda
conda activate BIOPYTHON

# run python script to get good contigs into one file
python ../scripts/filter_fasta_by_list_of_headers.py all.vib.contigs.fna good.vib.ids > good.vib.contigs.fna

    # actually the pathway changed to this??
    /home/csantosm/general_scripts/filter_fasta_by_list_of_headers.py
    # so try this?
    python /home/csantosm/general_scripts/filter_fasta_by_list_of_headers.py all.vib.contigs.fna good.vib.ids > good.vib.contigs.fna

    # 619 of the headers from list were not identified in the input fasta file
    # yeah there might have been some name changes, but whatever

# make another directory in drep folder for good contigs and move file to it
mkdir vibrant_good
mv ../vibrant/good.vib.contigs.fna vibrant_good

# separate the concatenated fasta file into multiple ones
# this can take awhile and you might get disconnected before it finishes, so use screen
awk '/^>/ {OUT=substr($0,2) ".fa"}; OUT {print >OUT}' *.fna 

# you can monitor how close it is to done by counting file names
ls -1 | wc -l


# make directory for split contigs within vibrant_good and move split contigs into this directory
mkdir split_contigs
mv *.fa split_contigs

# remove the concatenated file
rm good.vib.contigs.fna

# run script again, but with new folder
sbatch drep.sh vibrant_good

-----------------------------------------------------------------------------------------

# if this still doesn't work, split into smaller groups
cat vibrant_contigs/B* > blodgett.vib.contigs.fna

# to see how many contigs in file
grep '>' blodgett.vib.contigs.fna | wc -l
# 151,158 total just from B samples

cat vibrant_contigs/U* >> blodgett.vib.contigs.fna

# to make sure more were added
grep '>' blodgett.vib.contigs.fna | wc -l
# 327,006 total now, so yes, it was appended

# just get medium/high/circular quality drafts from blodgett subset
# load Chris's conda space
source /home/csantosm/initconda
conda activate BIOPYTHON
# run python script to get good contigs into one file
python /home/csantosm/general_scripts/filter_fasta_by_list_of_headers.py blodgett.vib.contigs.fna good.vib.ids > good.blodgett.vib.contigs.fna

# count how many good contigs
grep '>' good.blodgett.vib.contigs.fna | wc -l
# 91,043 (91,043/151,158 = ~60%)

# make another directory in drep folder for good contigs and move file to it
mkdir vibrant_good_blodgett
mv ../vibrant/good.blodgett.vib.contigs.fna vibrant_good_blodgett

# separate the concatenated fasta file into multiple ones
# this can take awhile and you might get disconnected before it finishes, so use screen
awk '/^>/ {OUT=substr($0,2) ".fa"}; OUT {print >OUT}' *.fna 

# you can monitor how close it is to done by counting file names
ls -1 | wc -l


# make directory for split contigs within vibrant_good and move split contigs into this directory
mkdir split_contigs
mv *.fa split_contigs

# remove the concatenated file
rm good.blodgett.vib.contigs.fna

# run script again, but with new folder
sbatch drep.sh vibrant_good_blodgett
# this got killed because OUT OF MEMORY

-----------------------------------------------------------------------------------------

# if this still doesn't work, split into even smaller groups
cat vibrant_contigs/B* > burn.vib.contigs.fna

# to see how many contigs in file
grep '>' burn.vib.contigs.fna | wc -l
# 151,158 total just from B samples

# split more
cat vibrant_contigs/BS* > burn.surface.vib.contigs.fna
cat vibrant_contigs/BD* > burn.deep.vib.contigs.fna

grep '>' burn.surface.vib.contigs.fna | wc -l
# 73,892 total from BS samples
grep '>' burn.deep.vib.contigs.fna | wc -l
# 77,266 total from BD samples

cat vibrant_contigs/U* > control.vib.contigs.fna

# to see how many contigs in file
grep '>' control.vib.contigs.fna | wc -l
# 175,848 total just from U samples

grep '>' control.surface.vib.contigs.fna | wc -l
# 90,491 total from US samples
grep '>' control.deep.vib.contigs.fna | wc -l
# 85,357 total from UD samples

# just get medium/high/circular quality drafts from blodgett subset
# load Chris's conda space
source /home/csantosm/initconda
conda activate BIOPYTHON
# run python script to get good contigs into one file
python /home/csantosm/general_scripts/filter_fasta_by_list_of_headers.py burn.vib.contigs.fna good.vib.ids > good.burn.vib.contigs.fna
python /home/csantosm/general_scripts/filter_fasta_by_list_of_headers.py control.vib.contigs.fna good.vib.ids > good.control.vib.contigs.fna
python /home/csantosm/general_scripts/filter_fasta_by_list_of_headers.py control.surface.vib.contigs.fna good.vib.ids > good.control.surface.vib.contigs.fna
python /home/csantosm/general_scripts/filter_fasta_by_list_of_headers.py control.deep.vib.contigs.fna good.vib.ids > good.control.deep.vib.contigs.fna

python /home/csantosm/general_scripts/filter_fasta_by_list_of_headers.py burn.surface.vib.contigs.fna good.vib.ids > good.burn.surface.vib.contigs.fna

python /home/csantosm/general_scripts/filter_fasta_by_list_of_headers.py burn.deep.vib.contigs.fna good.vib.ids > good.burn.deep.vib.contigs.fna

# count how many good contigs
grep '>' good.burn.vib.contigs.fna | wc -l
# 42,171 (42,171/151,158 = ~28%)
grep '>' good.burn.surface.vib.contigs.fna | wc -l
# 21,119 (21,119/73,892 = ~29%)
grep '>' good.burn.deep.vib.contigs.fna | wc -l
# 21,052 (21,052/77,266 = ~28%)
grep '>' good.control.vib.contigs.fna | wc -l
# 48,872 (48,872/175,848 = ~28%)
grep '>' good.control.surface.vib.contigs.fna | wc -l
# 26,221 (26,221/90,491 = ~29%)
grep '>' good.control.deep.vib.contigs.fna | wc -l
# 22,651 (22,651/85,357 = ~27%)

# make another directory in drep folder for good contigs and move file to it
mkdir vibrant_good_burn
mv ../vibrant/good.burn.vib.contigs.fna vibrant_good_burn
mkdir vibrant_good_burn_surface
mv ../vibrant/good.burn.surface.vib.contigs.fna vibrant_good_burn_surface
mkdir vibrant_good_burn_deep
mv ../vibrant/good.burn.deep.vib.contigs.fna vibrant_good_burn_deep


mkdir vibrant_good_control
mv ../vibrant/good.control.vib.contigs.fna vibrant_good_control
mkdir vibrant_good_control_surface
mv ../vibrant/good.control.surface.vib.contigs.fna vibrant_good_control_surface
mkdir vibrant_good_control_deep
mv ../vibrant/good.control.deep.vib.contigs.fna vibrant_good_control_deep

# separate the concatenated fasta file into multiple ones
# this can take awhile and you might get disconnected before it finishes, so use screen
screen
awk '/^>/ {OUT=substr($0,2) ".fa"}; OUT {print >OUT}' *.fna 

# you can monitor how close it is to done by counting file names
ls -1 | wc -l


# make directory for split contigs within vibrant_good and move split contigs into this directory
mkdir split_contigs
mv *.fa split_contigs

# remove the concatenated file
rm good.burn.vib.contigs.fna
rm good.control.vib.contigs.fna

# run script again, but with new folder
sbatch drep.sh vibrant_good_burn
# this one got cancelled due to time limit

sbatch drep.sh vibrant_good_control
# this one got fucked up, I don't know why

sbatch drep.sh vibrant_good_control_deep
# this one worked

sbatch drep.sh vibrant_good_control_surface
# this one worked

-----------------------------------------------------------------------------------------

cat vibrant_contigs/M* > lnu.vib.contigs.fna

# to see how many contigs in file
grep '>' lnu.vib.contigs.fna | wc -l
# 40,003 total just from M samples

cat vibrant_contigs/Q* >> lnu.vib.contigs.fna
# to make sure more were added
grep '>' lnu.vib.contigs.fna | wc -l
# 77,433 total now, so yes, it was appended

# just get medium/high/circular quality drafts from LNU subset
# load Chris's conda space
source /home/csantosm/initconda
conda activate BIOPYTHON
# run python script to get good contigs into one file
python /home/csantosm/general_scripts/filter_fasta_by_list_of_headers.py lnu.vib.contigs.fna good.vib.ids > good.lnu.vib.contigs.fna

python /home/csantosm/general_scripts/filter_fasta_by_list_of_headers.py c.vib.contigs.fna good.vib.ids > good.c.vib.contigs.fna

python /home/csantosm/general_scripts/filter_fasta_by_list_of_headers.py hn.vib.contigs.fna good.vib.ids > good.hn.vib.contigs.fna

python /home/csantosm/general_scripts/filter_fasta_by_list_of_headers.py hd.vib.contigs.fna good.vib.ids > good.hd.vib.contigs.fna

# count how many good contigs
grep '>' good.lnu.vib.contigs.fna | wc -l
# 15,391 (15,391/77,433 = ~20%)

grep '>' good.c.vib.contigs.fna | wc -l
#9,318

grep '>' good.hn.vib.contigs.fna | wc -l
#8,911

grep '>' good.hd.vib.contigs.fna | wc -l
#18,786

# make another directory in drep folder for good contigs and move file to it
mkdir vibrant_good_lnu
mv ../vibrant/good.lnu.vib.contigs.fna vibrant_good_lnu

# separate the concatenated fasta file into multiple ones
# this can take awhile and you might get disconnected before it finishes, so use screen
awk '/^>/ {OUT=substr($0,2) ".fa"}; OUT {print >OUT}' *.fna 

# you can monitor how close it is to done by counting file names
ls -1 | wc -l

# make directory for split contigs within vibrant_good and move split contigs into this directory
mkdir split_contigs
mv *.fa split_contigs

# run script again, but with new folder
sbatch drep.sh vibrant_good_lnu

-----------------------------------------------------------------------------------------

# when done:
# move into dereplicated_genomes folder and calculate number of vOTUs
# this is the total number of dereplicated vOTUs
cd dereplicated_genomes
ls -1 | wc -l

# lnu - 13,709 dereplicated genomes
# blodgett burn surface plots - 17,083
# blodgett burn deep plots - 17,688
    # blodgett burn total - 27,763
# blodgett control surface plots - 21,939
# blodgett control deep plots - 19,312
    # blodgett control total - 34,706
    
# DEREPLICATION BURN + CONTROL (input of 62,469) FAILED (unstacked dataframe is too big, causing int32 overflow)
# all contigs from these two groups are in fire/drep/blodgett/split_contigs
    
# heat dereplicated genomes
# control - 2,752
# dnase - 3,995
# no dnase - 2,065
# all good - 8,812 --> 4,644


# move  into split_contigs folders to find original number of vOTUs
cd split_contigs
ls -1 | wc -l

# YOU HAVE TO COMBINE SUB GROUPS INTO BIGGER GROUPS AND RUN DEREPLICATION AGAIN 
# UNTIL YOU HAVE COMPARED ALL DEREPLICATED GENOMES TO EACH OTHER

cat burn/burn_dRep/dereplicated_genomes/*.fa > blodgett/blodgett.vib.contigs.fna
cat control/control/_dRep/dereplicated_genomes/*.fa >> blodgett/blodgett.vib.contigs.fna

cat control/control_dRep/dereplicated_genomes/*.fa > allgood/allgood.vib.contigs.fna
cat dnase/dnase_dRep/dereplicated_genomes/*.fa >> allgood/allgood.vib.contigs.fna
cat nodnase/nodnase_dRep/dereplicated_genomes/*.fa >> allgood/allgood.vib.contigs.fna

# downloading csv tables (you need to be signed out)

rsync -r --progress seuge91@farm:~/blodgett/Project_JESG_Nova348P_Geonczy/drep/all/all_dRep/data_tables/genomeInformation.csv /mnt/c/Users/segeo/Downloads
rsync -r --progress seuge91@farm:~/fire/drep/vibrant_good/vibrant_good/_dRep/data_tables/genomeInformation.csv /mnt/c/Users/segeo/Downloads

In [ ]:
~/fire/drep/blodgett/lukedrep

# Dereplication alternative
# concatenated file of all viral contigs
# run megablast on all contigs (the one concatenated file) - raw results - for each contig, it'll run a blast comparison search for all the other contigs to show how similar they are
# megablast is tolerant of gaps (not just pair-wise alignment that CDHIT does)
# two python scripts - one calculates average nucleotide identity (anicalc.py), the other clusters (aniclust.py)
# folder with file of interest, snakemake file, .sh, qsubfolder "scripts", scripts subfolder has anicalc.py and aniclust.py, 
# how is this different than drep? a little less sensitive?
# leiden clustering algorithm - used for IMGVR v4 - mine NCBI for viral sequences - all sequenced viral sequences - clustered, all vOTUs
# 327006 --> 222993
-----------------------------------------------
# Introduction

# This stage combined all viral contigs previously identified in all samples 
# beginning with B or U into one file (BU.fna) and then used the clustering method used 
# in IMG/VR v4 to dereplicate the dataset.

# Conda env
# cluster-leiden - originally built to run snakemake and then had other dependencies added in

# Files
# BU.fna - combined fasta file input
# BU_blast.tsv - BLAST results file
# BU_ani.tsv - ANI results file
# BU_clusters.tsv - cluster file
# BU_clustered-leiden.fna - clustered fasta file output

# cluster.sh - bash script that explicitly specifies settings used by IMG/VR v4/ accepted by community. 
# These are also generally the default values
# clustering_slurm.sh - SLURM submission script. More cores = faster run time
# contig-ani-leiden-clustering-pipeline.smk - snakefile that runs the pipeline
# scripts/anicalc.py - Calculates ani values for results from megablast
# scripts/aniclust.py - Performs clustering based on ani values. 
    # This step is slower than the algorithm used in IMG/VR v3 but reportedly more accurate (goes over my head a bit)

# Dependencies
snakemake
numpy
igraph
BLAST
seqkit

# Notes

# anicalc.py and aniclust.py need to be in a scripts folder within the folder containing the snakefile. 
# At some point I will look into making other locations possible

 

# Refs
# Github - https://github.com/apcamargo/bioinformatics-snakemake-pipelines/tree/main/contig-ani-leiden-clustering-pipeline
# Paper - https://pubmed.ncbi.nlm.nih.gov/36399502/


In [ ]:
mkdir scripts
nano anicalc.py
-----------------------------------------------------------------------------------------

from collections import namedtuple

Hsp = namedtuple(
    "Hsp",
    ["qname", "tname", "pid", "len", "qcoords", "tcoords", "evalue", "qlen", "tlen"],
)


def parse_blast(handle):
    for line in handle:
        line = line.split()
        yield Hsp(
            qname=line[0],
            tname=line[1],
            pid=float(line[2]) / 100,
            len=float(line[3]),
            qcoords=sorted([int(line[4]), int(line[5])]),
            tcoords=sorted([int(line[6]), int(line[7])]),
            evalue=float(line[8]),
            qlen=float(line[9]),
            tlen=float(line[10]),
        )


def yield_alignment_blocks(handle):
    # init block with 1st record
    key, alns = None, None
    for aln in parse_blast(handle):
        if aln.qname == aln.tname:
            continue
        key = (aln.qname, aln.tname)
        alns = [aln]
        break
    # loop over remaining records
        for aln in parse_blast(handle):
        # skip self hits
        if aln.qname == aln.tname:
            continue
        # extend block
        elif (aln.qname, aln.tname) == key:
            alns.append(aln)
        # yield block and start new one
        else:
            yield alns
            key = (aln.qname, aln.tname)
            alns = [aln]
    yield alns


def prune_alns(alns, max_evalue=snakemake.params.blast_max_evalue):
    # remove short aligns
    alns = [aln for aln in alns if aln.evalue <= max_evalue]
    return alns


def compute_ani(alns):
    return round(sum(a.len * (a.pid) for a in alns) / sum(a.len for a in alns), 4)


def compute_cov(alns):
    # merge qcoords
    coords = sorted([a.qcoords for a in alns])
    nr_coords = [coords[0]]
    for start, stop in coords[1:]:
        # overlapping, update start coord
        if start <= (nr_coords[-1][1] + 1):
            nr_coords[-1][1] = max(nr_coords[-1][1], stop)
        # non-overlapping, append to list
                else:
            nr_coords.append([start, stop])
    # compute qry_cov
    alen = sum([stop - start + 1 for start, stop in nr_coords])
    qcov = round(alen / alns[0].qlen, 4)

    # merge tcoords
    coords = sorted([a.tcoords for a in alns])
    nr_coords = [coords[0]]
    for start, stop in coords[1:]:
        # overlapping, update start coord
        if start <= (nr_coords[-1][1] + 1):
            nr_coords[-1][1] = max(nr_coords[-1][1], stop)
        # non-overlapping, append to list
        else:
            nr_coords.append([start, stop])
    # compute qry_cov
    alen = sum(stop - start + 1 for start, stop in nr_coords)
    tcov = round(alen / alns[0].tlen, 4)
    return qcov, tcov


out = open(snakemake.output[0], "w")
out.write("\t".join(["contig_1", "contig_2", "num_alns", "ani", "qcov", "tcov"]) + "\n")
input = open(snakemake.input[0])
for alns in yield_alignment_blocks(input):
    alns = prune_alns(alns)
    if len(alns) == 0:
        continue
    qname, tname = alns[0].qname, alns[0].tname
    ani = compute_ani(alns)
    qcov, tcov = compute_cov(alns)
    row = [qname, tname, len(alns), ani, qcov, tcov]
    out.write("\t".join(str(_) for _ in row) + "\n")
input.close()
out.close()

-----------------------------------------------------------------------------------------

nano aniclust.py

-----------------------------------------------------------------------------------------

from collections import defaultdict

import itertools
import random
import igraph as ig
import numpy as np


class Sequence:
    def __init__(self, header, seq):
        self._header = header
        self._seq = seq

    @property
    def header(self):
        return self._header

    @property
    def id(self):
        return self._header.split()[0]

    @property
    def seq(self):
        return self._seq

    def count(self, substring):
        return self.seq.count(substring)

    def __len__(self):
        return len(self.seq)


def read_fasta(filepath):
    with open(filepath) as fin:
        last = None
        while True:
            if not last:
                for l in fin:
                    if l[0] == ">":
                        last = l[:-1]
                        break
            if not last:
                break
            name, seqs, last = last[1:], [], None
            for l in fin:
                if l[0] == ">":
                    last = l[:-1]
                    break
                seqs.append(l[:-1])
            seqs = "".join(seqs)
            if len(seqs):
                yield Sequence(name, seqs)
            if not last:
                break


def build_graph(
    ani_file,
    fasta_file,
    min_ani=snakemake.params.min_ani,
    min_cov=snakemake.params.min_cov,
):
    # Read the input FASTA file to get the names of all the sequences (nodes)
    nodes = []
    lengths = []
    for seq in read_fasta(fasta_file):
        nodes.append(seq.id)
        lengths.append(len(seq) - seq.seq.upper().count("N"))   
    nodes_set = set(nodes)
    # Initiate the lists that will store the edges between two sequences and
    # their respective weights (ANI × coverage) and ANI
    edges = []
    weights = defaultdict(list)
    anis = defaultdict(list)
    with open(ani_file) as fin:
        # Skip header
        next(fin)
        for line in fin:
            (
                seq_1,
                seq_2,
                _,
                ani,
                qcov,
                tcov,
            ) = line.strip().split("\t")
            ani, qcov, tcov = (
                float(ani),
                float(qcov),
                float(tcov),
            )
            # If both the sequences in the ANI pair were in the input FASTA file,
            # their ANI is equal to or greater than `min_ani`, and either`qcov`
            # or `tcov` is equal to or greater than `min_cov`, store their connection
            if (
                ((seq_1 in nodes_set) and (seq_2 in nodes_set))
                and (ani >= min_ani)
                and ((qcov >= min_cov) or (tcov >= min_cov))
            ):
                pair = tuple(sorted([seq_1, seq_2]))
                edges.append(pair)
                anis[pair].append(ani)
                weight = ani * max([qcov, tcov])
                weights[pair].append(weight)
    # Take the mean ANI and weight for each pair of sequences
    anis = [np.mean(anis[pair]) for pair in edges]
    weights = [np.mean(weights[pair]) for pair in edges]
    # Create a graph from the node list and add edges weighted by the ANI between
    # the sequences being connected
    graph = ig.Graph()
    graph.add_vertices(list(nodes), attributes={"length": lengths})
    graph.add_edges(edges, attributes={"weight": weights, "ani": weights})
    return graph


def pick_resolution(
    graph,
    target_avg_ani=snakemake.params.avg_ani,
    steps=101,
    seed=snakemake.params.seed,
):
    """
    Given a graph (`graph`) and a target average within-cluster ANI (`target_avg_ani`),
    get the resolution parameter that will provide the closest within-community average
    edge weight using the Leiden clustering algorithm.
    """
    # The `last_res` variable will store the resolution (`res`) value of the previous
    # iteration, while the `last_avg_weight` will store the difference between the average
    # AAI of the previous iteration and the target average ANI (`target_avg_ani`)
    last_res = None
    last_avg_weight = None
    # Iterate through resolution parameters (`res`)
    for res in np.linspace(2, 0, steps):
        random.seed(seed)
        # Find the communities using the current resolution parameter
        communities = graph.community_leiden(weights="weight", resolution_parameter=res)
        # Initiate the list that will store the edges' ANI
        ani_list = []
        # Iterate through the communities subgraphs ordered by length (largest → smallest)
        for sg in reversed(np.argsort(communities.sizes())):
            members = [i.attributes()["name"] for i in communities.subgraph(sg).vs]
            # If the community has a single member, break the loop
            if len(members) < 2:
                break
            # Add all the pairwise ANI value to the `ani_list` list
            for i, j in itertools.combinations(members, 2):
                try:
                    edge = graph.get_eid(i, j)
                    edge = graph.es[edge]
                    ani_list.append(edge.attributes()["ani"])
                # In case two nodes are not connected, skip the pair
                except Exception:
                    continue
        # Compute the average value of all the edges in the iteration
        current_avg_ani = np.mean(ani_list) if len(ani_list) else 1
        # If this is the first iteration (that is, `res == 1`)
        if res == 2:
            last_res = res
            last_avg_weight = current_avg_ani
        # If the difference between the curent iteration average ANI and the target ANI
        # is less than the difference of the previous iteration, store the value and keep
        # iterating through `res`
        elif np.abs(target_avg_ani - current_avg_ani) <= np.abs(
            target_avg_ani - last_avg_weight
        ):
            last_res = res
            last_avg_weight = current_avg_ani
        # If the difference between the curent iteration average ANI and the target ANI
        # is greater than the last iteration, break the loop
        else:
            break
    return last_res


ani_graph = build_graph(snakemake.input.ANI, snakemake.input.FASTA)
if snakemake.params.leiden_resolution == "auto":
    leiden_resolution = pick_resolution(ani_graph)
else:
    leiden_resolution = snakemake.params.leiden_resolution
clusters = ani_graph.community_leiden(
    weights="weight", resolution_parameter=leiden_resolution
)

with open(snakemake.output[0], "w") as fout:
    for i in reversed(np.argsort(clusters.sizes())):
        subgraph = clusters.subgraph(i)
        members = {v.attributes()["name"]: v.attributes()["length"] for v in subgraph.vs}
        members = sorted(members, key=members.get, reverse=True)
        fout.write(members[0] + "\t" + ",".join(members) + "\n")

# Read Mapping

In [ ]:
# make the directory structure for bowtie
mkdir bowtie
cd bowtie
mkdir err log alignments ref coverm

# concatenate all the dereplicated vOTUs into a single fasta file and save the new file in the bowtie folder.

# the name of the new database file is all.drep.contigs.fa (updated_heat)

cat /home/seuge91/heat/updated_drep/all/all_dRep/dereplicated_genomes/* >
/home/seuge91/heat/updated_bowtie/all.drep.contigs.fa

# the name of the new database file is all.good.drep.contigs.fa (fire)

cat /home/seuge91/fire/drep/all_good/all_good/_dRep/dereplicated_genomes/* >
/home/seuge91/fire/bowtie/all.good.drep.contigs.fa

cat /home/seuge91/heat/drep/allgood/allgood_dRep/dereplicated_genomes/* > /home/seuge91/heat/bowtie/allgood.drep.contigs.fa

# make bowtie reference script
# 
cd /home/seuge91/fire/scripts
nano bowtie_ref.sh

cd /home/seuge91/heat/scripts
nano updated_bowtie_ref.sh

----------------------------

#!/bin/bash
#SBATCH --job-name=bt2ref
#SBATCH --nodes=1
#SBATCH -t 12:00:00
#SBATCH --ntasks=1
#SBATCH --partition=bmh

# for calculating the amount of time the job takes
begin=`date +%s`
echo $HOSTNAME

module load bowtie2

cd /home/seuge91/fire/bowtie/ref/

bowtie2-build ../all.good.drep.contigs.fa all_good_vibrant_drep

# getting end time to calculate time elapsed
end=`date +%s`
elapsed=`expr $end - $begin`
echo Time taken: $elapsed
    
----------------------------

#!/bin/bash
#SBATCH --job-name=bt2ref
#SBATCH --nodes=1
#SBATCH -t 12:00:00
#SBATCH --ntasks=1
#SBATCH --partition=bmh

# for calculating the amount of time the job takes
begin=`date +%s`
echo $HOSTNAME

module load bowtie2

cd /home/seuge91/heat/bowtie/ref/

bowtie2-build ../allgood.drep.contigs.fa allgood_vibrant_drep

# getting end time to calculate time elapsed
end=`date +%s`
elapsed=`expr $end - $begin`
echo Time taken: $elapsed
    
----------------------------

# updated_bowtie_ref.sh
----------------------------

#!/bin/bash
#SBATCH --job-name=bt2ref
#SBATCH --nodes=1
#SBATCH -t 12:00:00
#SBATCH --ntasks=1
#SBATCH --partition=bmh

# for calculating the amount of time the job takes
begin=`date +%s`
echo $HOSTNAME

module load bowtie2

cd /home/seuge91/heat/updated_bowtie/ref/

bowtie2-build ../all.drep.contigs.fa updated_all_vibrant_drep

# getting end time to calculate time elapsed
end=`date +%s`
elapsed=`expr $end - $begin`
echo Time taken: $elapsed
    
----------------------------

# run the script to index the reference database
cd /home/seuge91/fire/scripts
sbatch --output=../bowtie/log/ref.log --error=../bowtie/err/ref.err bowtie_ref.sh

cd /home/seuge91/fire/scripts
sbatch --output=../lnu_bowtie/log/ref.log --error=../lnu_bowtie/err/ref.err lnu_bowtie_ref.sh

cd /home/seuge91/fire/scripts
sbatch --output=../burn_bowtie/log/ref.log --error=../burn_bowtie/err/ref.err burn_bowtie_ref.sh

cd /home/seuge91/heat/scripts
sbatch --output=../bowtie/log/ref.log --error=../bowtie/err/ref.err bowtie_ref.sh

cd /home/seuge91/heat/scripts
sbatch --output=../updated_bowtie/log/ref.log --error=../bowtie/err/ref.err updated_bowtie_ref.sh



# map the reads against the database
# make mapping script
cd /home/seuge91/fire/scripts
nano bowtie_map.sh

cd /home/seuge91/heat/scripts
nano updated_bowtie_map.sh

----------------------------

#!/bin/bash
#SBATCH --job-name=bt2map
#SBATCH --nodes=1
#SBATCH -t 2:00:00
#SBATCH --ntasks=48
#SBATCH --partition=bmh

# for calculating the amount of time the job takes
begin=`date +%s`
echo $HOSTNAME

#load modules
module load bowtie2
module load samtools

path=/home/seuge91/heat/
sample=${1}

cd /home/seuge91/heat/updated_bowtie/ref/

bowtie2 -x updated_all_vibrant_drep -p 48 \
-1 ${path}reads/rmphix/${sample}_R1_rmphix.fq.gz \
-2 ${path}reads/rmphix/${sample}_R2_rmphix.fq.gz \
-S ${path}updated_bowtie/alignments/${sample}.vib.sam \
--sensitive

cd /home/seuge91/heat/updated_bowtie/alignments
samtools view -F 4 -bS ${sample}.vib.sam | samtools sort > ${sample}.vib.sI.bam
samtools index ${sample}.vib.sI.bam

rm ${sample}.vib.sam

# getting end time to calculate time elapsed
end=`date +%s`
elapsed=`expr $end - $begin`
echo Time taken: $elapsed

----------------------------

#!/bin/bash
#SBATCH --job-name=bt2map
#SBATCH --nodes=1
#SBATCH -t 2:00:00
#SBATCH --ntasks=48
#SBATCH --partition=bmh

# for calculating the amount of time the job takes
begin=`date +%s`
echo $HOSTNAME

#load modules
module load bowtie2
module load samtools

path=/home/seuge91/fire/
sample=${1}

cd /home/seuge91/fire/bowtie/ref/

bowtie2 -x all_good_vibrant_drep -p 48 \
-1 ${path}reads/rmphix_combined/${sample}_R1_rmphix_combined.fq.gz \
-2 ${path}reads/rmphix_combined/${sample}_R2_rmphix_combined.fq.gz \
-S ${path}bowtie/alignments/${sample}.vib.sam \
--sensitive

cd /home/seuge91/fire/bowtie/alignments
samtools view -F 4 -bS ${sample}.vib.sam | samtools sort > ${sample}.vib.sI.bam
samtools index ${sample}.vib.sI.bam

rm ${sample}.vib.sam

# getting end time to calculate time elapsed
end=`date +%s`
elapsed=`expr $end - $begin`
echo Time taken: $elapsed
    
----------------------------

#!/bin/bash
#SBATCH --job-name=bt2map
#SBATCH --nodes=1
#SBATCH -t 2:00:00
#SBATCH --ntasks=48
#SBATCH --partition=bmh

# for calculating the amount of time the job takes
begin=`date +%s`
echo $HOSTNAME

#load modules
module load bowtie2
module load samtools

path=/home/seuge91/heat/
sample=${1}

cd /home/seuge91/heat/bowtie/ref/

bowtie2 -x allgood_vibrant_drep -p 48 \
-1 ${path}reads/rmphix/${sample}_R1_rmphix.fq.gz \
-2 ${path}reads/rmphix/${sample}_R2_rmphix.fq.gz \
-S ${path}bowtie/alignments/${sample}.vib.sam \
--sensitive

cd /home/seuge91/heat/bowtie/alignments
samtools view -F 4 -bS ${sample}.vib.sam | samtools sort > ${sample}.vib.sI.bam
samtools index ${sample}.vib.sI.bam

rm ${sample}.vib.sam

# getting end time to calculate time elapsed
end=`date +%s`
elapsed=`expr $end - $begin`
echo Time taken: $elapsed
    
----------------------------

# run mapping script
cd /home/seuge91/fire/scripts

for sample in $(<../sample.txt)
do
  sbatch --output=../bowtie/log/${sample}.map.log --error=../bowtie/err/${sample}.map.err bowtie_map.sh $sample
done


for sample in $(<../lnu_sample.txt)
do
  sbatch --output=../lnu_bowtie/log/${sample}.map.log --error=../lnu_bowtie/err/${sample}.map.err lnu_bowtie_map_tmp.sh $sample
done

for sample in $(<../burn_sample.txt)
do
  sbatch --output=../burn_bowtie/log/${sample}.map.log --error=../burn_bowtie/err/${sample}.map.err burn_bowtie_map.sh $sample
done

cd /home/seuge91/heat/scripts

for sample in $(<../sampleIDs.txt)
do
  sbatch --output=../updated_bowtie/log/${sample}.map.log --error=../updated_bowtie/err/${sample}.map.err updated_bowtie_map.sh $sample
done

# Generate vOTU table

In [ ]:
# make coverM script
nano coverm_tmp.sh
nano coverm.sh

nano updated_coverm.sh
----------------------------

#!/bin/bash
#SBATCH --job-name=coverm
#SBATCH --nodes=1
#SBATCH -t 10:00:00
#SBATCH --ntasks=1
#SBATCH --partition=bmh

# for calculating the amount of time the job takes
begin=`date +%s`
echo $HOSTNAME

## you need to run the command from my coverm folder
cd /home/csantosm/software/coverm-x86_64-unknown-linux-musl-0.6.1/

path=/home/seuge91/heat/updated_bowtie/

./coverm contig -m trimmed_mean --min-covered-fraction 0.75 -b ${path}/alignments/*.bam > ${path}/coverm/all.75.tmean.tsv

# getting end time to calculate time elapsed
end=`date +%s`
elapsed=`expr $end - $begin`
echo Time taken: $elapsed
    
----------------------------
----------------------------

#!/bin/bash
#SBATCH --job-name=coverm
#SBATCH --nodes=1
#SBATCH -t 10:00:00
#SBATCH --ntasks=1
#SBATCH --partition=bmh

# for calculating the amount of time the job takes
begin=`date +%s`
echo $HOSTNAME

## you need to run the command from my coverm folder
cd /home/csantosm/software/coverm-x86_64-unknown-linux-musl-0.6.1/

path=/home/seuge91/fire/bowtie/

./coverm contig -m trimmed_mean --min-covered-fraction 0.75 -b ${path}/alignments/*.bam > ${path}/coverm/all.good.75.tmean.tsv

# getting end time to calculate time elapsed
end=`date +%s`
elapsed=`expr $end - $begin`
echo Time taken: $elapsed
    
----------------------------

#!/bin/bash
#SBATCH --job-name=coverm
#SBATCH --nodes=1
#SBATCH -t 10:00:00
#SBATCH --ntasks=1
#SBATCH --partition=bmh

# for calculating the amount of time the job takes
begin=`date +%s`
echo $HOSTNAME

## you need to run the command from my coverm folder
cd /home/csantosm/software/coverm-x86_64-unknown-linux-musl-0.6.1/

path=/home/seuge91/heat/bowtie/

./coverm contig -m trimmed_mean --min-covered-fraction 0.75 -b ${path}/alignments/*.bam > ${path}/coverm/allgood.75.tmean.tsv

# getting end time to calculate time elapsed
end=`date +%s`
elapsed=`expr $end - $begin`
echo Time taken: $elapsed
    
----------------------------

# run script
cd /home/seuge91/fire/scripts
sbatch --output=../bowtie/log/coverm.log --output=../bowtie/err/coverm.err coverm_tmp.sh

cd /home/seuge91/fire/scripts
sbatch --output=../lnu_bowtie/log/coverm.log --output=../lnu_bowtie/err/coverm.err lnu_coverm_tmp.sh

cd /home/seuge91/fire/scripts
sbatch --output=../burn_bowtie/log/coverm.log --output=../burn_bowtie/err/coverm.err burn_coverm.sh

cd /home/seuge91/heat/scripts
sbatch --output=../bowtie/log/coverm.log --output=../bowtie/err/coverm.err coverm.sh

cd /home/seuge91/heat/scripts
sbatch --output=../updated_bowtie/log/coverm.log --output=../updated_bowtie/err/coverm.err updated_coverm.sh

# download table

scp seuge91@farm.cse.ucdavis.edu:/home/seuge91/fire/lnu_bowtie/coverm/*.tsv /mnt/c/Users/segeo/Downloads

scp seuge91@farm.cse.ucdavis.edu:/home/seuge91/fire/burn_bowtie/coverm/*.tsv /mnt/c/Users/segeo/Downloads

scp seuge91@farm.cse.ucdavis.edu:/home/seuge91/heat/bowtie/coverm/*.tsv /mnt/c/Users/segeo/Downloads

scp seuge91@farm.cse.ucdavis.edu:/home/seuge91/heat/updated_bowtie/coverm/*.tsv /mnt/c/Users/segeo/Downloads


# OK THIS PART STARTS TO GET SKETCHY...


# download table
rsync -r --progress seuge91@farm:~/blodgett/Project_JESG_Nova348P_Geonczy/bowtie2/sam/covm.75.tmean.tsv /mnt/c/Users/segeo/Downloads

    
cd /home/seuge91/fire/reads/
grep -e '#Total' *gz_stats.txt > cleanreadsbio.txt
mv cleanreadsbio.txt /home/jdfudyma/2203_pH/sam

cd /home/jdfudyma/2203_pH/remove_phix
grep -e '#Total' *gz_stats.txt > cleanreadstech.txt
mv cleanreadsbio.txt /home/jdfudyma/2203_pH/sam
```

6) Download coverage tables to your personal computer
```{bash}
#use scp to transfer
#make sure you ARE IN BASE TERMINAL (i.e. Janes-MBP) not logged into farm

#path to things on farm, path to where on your computer you want things 

#getting coverage tables
scp jdfudyma@farm.cse.ucdavis.edu:/home/jdfudyma/2203_pH/sam/*.tsv ~/Documents/CA/UC_Davis/Emerson_Lab/21.pH

#getting clean reads files
scp jdfudyma@farm.cse.ucdavis.edu:/home/jdfudyma/2203_pH/sam/*.txt ~/Documents/CA/UC_Davis/Emerson_Lab/21.pH
    
scp seuge91@farm.cse.ucdavis.edu:/home/seuge91/fire/lnu_bowtie/coverm/*.tsv /mnt/c/Users/segeo/Downloads

# Generating Count Table for Differential Abundance

In [ ]:
# make covermcount script
nano updated_covermcount.sh

----------------------------

#!/bin/bash
#SBATCH --job-name=covermcount
#SBATCH --nodes=1
#SBATCH -t 10:00:00
#SBATCH --ntasks=1
#SBATCH --partition=bmh

# for calculating the amount of time the job takes
begin=`date +%s`
echo $HOSTNAME

## you need to run the command from my coverm folder
cd /home/csantosm/software/coverm-x86_64-unknown-linux-musl-0.6.1/

path=/home/seuge91/heat/updated_bowtie/

./coverm contig -m count -b ${path}/alignments/*.bam > ${path}/coverm/all.count.tsv

# getting end time to calculate time elapsed
end=`date +%s`
elapsed=`expr $end - $begin`
echo Time taken: $elapsed
    
----------------------------

# make covermcount script
nano covermcount.sh

----------------------------

#!/bin/bash
#SBATCH --job-name=covermcount
#SBATCH --nodes=1
#SBATCH -t 10:00:00
#SBATCH --ntasks=1
#SBATCH --partition=bmh

# for calculating the amount of time the job takes
begin=`date +%s`
echo $HOSTNAME

## you need to run the command from my coverm folder
cd /home/csantosm/software/coverm-x86_64-unknown-linux-musl-0.6.1/

path=/home/seuge91/heat/bowtie/

./coverm contig -m count -b ${path}/alignments/*.bam > ${path}/coverm/allgood.count.tsv

# getting end time to calculate time elapsed
end=`date +%s`
elapsed=`expr $end - $begin`
echo Time taken: $elapsed
    
----------------------------
    
# Download table
scp seuge91@farm.cse.ucdavis.edu:/home/seuge91/heat/bowtie/coverm/*.count.tsv /mnt/c/Users/segeo/Downloads
    
scp seuge91@farm.cse.ucdavis.edu:/home/seuge91/heat/bowtie/coverm/*.count.tsv /mnt/c/Users/segeo/Downloads
    
scp seuge91@farm.cse.ucdavis.edu:/home/seuge91/heat/updated_bowtie/coverm/*.count.tsv /mnt/c/Users/segeo/Downloads
    
    
scp /mnt/d/MT_6123968/* seuge91@farm.cse.ucdavis.edu:/home/seuge91/jane 
    
scp /mnt/d/MT_6123968 jdfudyma@farm.cse.ucdavis.edu:/home/jdfudyma/2305_spatial/metabolomes 
    
chmod777 jane

In [ ]:
#!/bin/bash
#SBATCH --job-name=coverm
#SBATCH --nodes=1
#SBATCH -t 10:00:00
#SBATCH --ntasks=1
#SBATCH --partition=high2

# for calculating the amount of time the job takes
begin=`date +%s`
echo $HOSTNAME

## you need to run the command from my coverm folder
cd /home/csantosm/software/coverm-x86_64-unknown-linux-musl-0.6.1/

path=/home/seuge91/fire/lnu_bowtie/

./coverm contig -m trimmed_mean --min-covered-fraction 0.75 -b ${path}/alignments/*.bam > ${path}/coverm/lnu.good.75.tmean.tsv

# getting end time to calculate time elapsed
end=`date +%s`
elapsed=`expr $end - $begin`
echo Time taken: $elapsed

----------------------------

#!/bin/bash
#SBATCH --job-name=coverm
#SBATCH --nodes=1
#SBATCH -t 10:00:00
#SBATCH --ntasks=1
#SBATCH --partition=high2

# for calculating the amount of time the job takes
begin=`date +%s`
echo $HOSTNAME

## you need to run the command from my coverm folder
cd /home/csantosm/software/coverm-x86_64-unknown-linux-musl-0.6.1/
squeu
path=/home/seuge91/fire/burn_bowtie/

./coverm contig -m trimmed_mean --min-covered-fraction 0.75 -b ${path}/alignments/*.bam > ${path}/coverm/burn.good.75.tmean.tsv

# getting end time to calculate time elapsed
end=`date +%s`
elapsed=`expr $end - $begin`
echo Time taken: $elapsed

# Host Prediction

In [ ]:
#!/bin/bash
#SBATCH -D /home/seuge91/heat/scripts/
#SBATCH --job-name=iphop_split
#SBATCH --nodes=1
#SBATCH -t 1:00:00
#SBATCH --ntasks=1
#SBATCH --partition=bmm

# for calculating the amount of time the job takes
begin=`date +%s`
echo $HOSTNAME

source /home/csantosm/initconda
conda activate IPHOP

set=${1}

full_db=/home/seuge91/heat/updated_bowtie/all.drep.contigs.fa
split_db=/home/seuge91/heat/iphop/split_db

iphop split --input_file ${full_db} --split_dir ${split_db}

#getting end time to calculate time elapsed
end=`date +%s`
elapsed=`expr $end - $begin`
echo Time taken: $elapsed

----------------------------

#!/bin/bash
#SBATCH -D /home/seuge91/heat/scripts/
#SBATCH --job-name=iphop_predict
#SBATCH --nodes=1
#SBATCH -t 24:00:00
#SBATCH --mem=128GB
#SBATCH --ntasks=24
#SBATCH --partition=bmm

# for calculating the amount of time the job takes
begin=`date +%s`
echo $HOSTNAME

source /home/csantosm/initconda
conda activate IPHOP

batch=${1}

iphop_db=/home/csantosm/databases/IPHOP_db/Sept_2021_pub

cd /home/seuge91/heat/iphop

iphop predict --fa_file ./split_db/${batch}.fna --out_dir ./results/${batch} --db_dir ${iphop_db} --num_threads 24

#getting end time to calculate time elapsed
end=`date +%s`
elapsed=`expr $end - $begin`
echo Time taken: $elapsed
    
----------------------------
    
cd /home/seuge91/heat/scripts

batches="batch_00001 batch_00002 batch_00003 batch_00004 batch_00005 batch_00006 batch_00007 batch_00008 batch_00009 batch_00010 batch_00011 batch_00012 batch_00013 batch_00014 batch_00015 batch_00016 batch_00017 batch_00018"
for batch in $batches
do
sbatch --output=../iphop/log/${batch}.log --error=../iphop/err/${batch}.err iphop_predict.sh $batch
done

batches="batch_00000"
for batch in $batches
do
sbatch --output=../iphop/log/${batch}.log --error=../iphop/err/${batch}.err iphop_predict.sh $batch
done

# combine results from batches
# get header from one of the batches
cd batch_00000
head -n 1 Host_prediction_to_genome_m90.csv > all_Host_prediction_to_genome_m90.csv
head -n 10 Detailed_output_by_tool.csv > all_Detailed_output_by_tool.csv
# move files up a folder
mv all_Host_prediction_to_genome_m90.csv ..
#get body of each batch and concatenate to main file
tail -n +2 -q batch_*/Host_prediction_to_genome_m90.csv >> all_Host_prediction_to_genome_m90.csv
tail -n +2 -q batch_*/Host_prediction_to_genus_m90.csv >> all_Host_prediction_to_genus_m90.csv
tail -n +11 batch_*/Detailed_output_by_tool.csv >> all_Detailed_output_by_tool.csv

# download files
scp seuge91@farm.cse.ucdavis.edu:/home/seuge91/heat/iphop/results/*.csv /mnt/c/Users/segeo/Downloads
    

# vcontact

In [ ]:
# make vcontact directory for vibrant contig proteins!!
mkdir vcontact

# within vcontact directory, make more directories
mkdir err log

# concatenate all viral contigs from drep into a single file
cat *.fa > allcontigs.drep.fa 

# count how many total contigs
grep -c contig allcontigs.drep.fa
#18,869

# move file into vcontact directory
mv allcontigs.drep.fa ../../../vcontact

# predict proteins 
#-i flag means input file, all contigs from drep 
#-a flag means output file that is protein, needs to be named with end of ".faa" 
 
source /home/csantosm/inticonda 
conda activate PRODIGAL 
prodigal -i allcontigs.drep.fa -a votus.prodigal.faa -p meta


# create gene2genome.csv file
# make script
nano gtg.sh

----------------------------

#!/bin/bash
#SBATCH -D /home/seuge91/heat/scripts/
#SBATCH --job-name=g2g
#SBATCH --nodes=1
#SBATCH -t 1:00:00
#SBATCH --partition=bmm

# for calculating the amount of time the job takes
begin=`date +%s`
echo $HOSTNAME

source /home/csantosm/initconda
conda activate VCONTACT2

cd /home/seuge91/heat/vcontact/

vcontact2_gene2genome -p ./votus.prodigal.faa  \
-o ./votus.gene2genome.csv \
-s Prodigal-FAA

# getting end time to calculate time elapsed
end=`date +%s`
elapsed=`expr $end - $begin`
echo Time taken: $elapsed

----------------------------

# run script
sbatch --output=../vcontact/log/g2g.log --error=../vcontact/err/g2g.err g2g.sh

# create vcontact script
cd /home/seuge91/heat/scripts/
nano vcontact

----------------------------

#!/bin/bash
#SBATCH -D /home/seuge91/heat/scripts/
#SBATCH --job-name=vcontact
#SBATCH --nodes=1
#SBATCH -t 48:00:00
#SBATCH --ntasks=16
#SBATCH --partition=bmm

# for calculating the amount of time the job takes
begin=`date +%s`
echo $HOSTNAME

source /home/csantosm/initconda
conda activate VCONTACT2
module load deprecated/java

cd /home/seuge91/heat/vcontact/

vcontact2 --raw-proteins votus.prodigal.faa \
--rel-mode 'Diamond' \
--db 'ProkaryoticViralRefSeq85-Merged' \
--proteins-fp votus.gene2genome.csv \
--pcs-mode MCL \
--vcs-mode ClusterONE \
--threads 16 \
--c1-bin /home/csantosm/miniconda3/bin/cluster_one-1.0.jar \
--output-dir vcontact_out

# getting end time to calculate time elapsed
end=`date +%s`
elapsed=`expr $end - $begin`
echo Time taken: $elapsed
    
----------------------------

# run script
sbatch --output=../vcontact/log/vc2.log --error=../vcontact/err/vc2.err vcontact.sh

# download files
scp seuge91@farm.cse.ucdavis.edu:/home/seuge91/heat/vcontact/vcontact_out/genome_by_genome_overview.csv /mnt/c/Users/segeo/Downloads
scp seuge91@farm.cse.ucdavis.edu:/home/seuge91/heat/vcontact/vcontact_out/c1.ntw /mnt/c/Users/segeo/Downloads

# Cleaning Up

In [ ]:
Removing intermediate files

# remove trimmed reads (anything before Phi X removal)
rm -r trimmed
rm -r unpaired

# if you have concatenated batch fastq.gz files, remove all the batches and just keep the combined concatenated files
rm -r rmphix
rm -r rmphix_unpaired

# KEEP rmphix_combined and rmphix_combined_unpaired

# remove intermediate files in megahit
cd /megahit
rm -r *V*/intermediate_contigs



# remove Mdb.csv in dereplication step
cd ../drep/all/all_dRep/data_tables

# explanation of what each file represents
cd vibrant_good/vibrant_good_dRep/data_tables
./data_tables
...../Bdb.csv  # Sequence locations and filenames
...../Cdb.csv  # Genomes and cluster designations
...../Chdb.csv # CheckM results for Bdb
...../Mdb.csv  # Raw results of MASH comparisons
...../Ndb.csv  # Raw results of ANIn comparisons
...../Sdb.csv  # Scoring information
...../Wdb.csv  # Winning genomes
...../Widb.csv # Winning genomes' checkM information

rm Mdb.csv

# SAM files (read mapping, bowtie)


# how much storage you are taking up
# make script

nano diskuse.sb

-----------------------------------------------------------------------------------------

#!/bin/bash
#SBATCH -D /home/seuge91
#SBATCH --job-name=DiskUse
#SBATCH --nodes=1
#SBATCH -t 12:00:00
#SBATCH --ntasks=8
#SBATCH --output=DiskUse_%j.out
#SBATCH --error=DiskUse_%j.err
#SBATCH --partition=med2

cd /home/seuge91
echo seuge91 >> /home/seuge91/DiskUse.txt
du -sh . >> /home/seuge91/DiskUse.txt

-----------------------------------------------------------------------------------------

sbatch diskuse.sb

# will create DiskUse.txt file and show how many TB you are using

# Preparing dataframe in R

In [ ]:
library(tidyverse)
otu=read_delim("lnu.good.75.tmean.tsv", delim = "\t", col_names = TRUE)
contignames = otu$Contig

otu=otu%>%
  select(-Contig)
otu=data.frame(otu)
row.names(otu)=contignames
colnames(otu) <- gsub(pattern=".vib.sI.Trimmed.Mean", replacement = "", x=colnames(otu))

library(vegan)

# relativized data set
# took every cell (OTUs abundance in particular sample) and divided it by the sum of every OTUs abundance in that sample = to give relative abundance
# decostand - data (otu table), type of standardization/relativization (total is relative abundance), margin says if we perform that across rows are column and 2 is equal to column
# colSums confirms that everything equals 1

otu.rel=decostand(otu, method = "total", MARGIN = 2)
colSums(otu.rel)

# PCoA plot

In [ ]:
# calculate pairwise distance between each pair of samples
# in a particular sample, how similar is this community to all of the other samples?

# bray-curtis takes into account both presence and relative abundance
# make a distance object
# t(otu.rel) because vegdist by default calculate pairwise distances between rows, but we wanted columns because this represents samples, so t transposes so that columns are rows

otu.dist=vegdist(t(otu.rel), method = "bray")

# distance matrix tells us how dissimilar every pair of samples are from each other
# this matrix has 38 dimensions of data represented, how far apart every samples is from every other sample
# so PCoA, collapses 38 axes into 2 axes to visualize 38 dimensions into 2 dimensions

otu.pcoa=cmdscale(otu.dist, eig = TRUE)
otu.pcoa$points
otu.pcoa.points=data.frame(otu.pcoa$points)
colnames(otu.pcoa.points)=c("pcoa1", "pcoa2")

otu.pcoa.points$location <- "McLaughlin"
otu.pcoa.points$location[grepl("QV", row.names(otu.pcoa.points))] <- "QuailRidge"

# make a PCoA plot!!

ggplot(otu.pcoa.points, aes(x=pcoa1, y=pcoa2, color=location))+
  geom_point()
    
# make another file with one column in common with name the same
# sample 1 is called the exact same way in both tables and you can join both dataframes by that column

write.table(x=otu.pcoa.points, file = "lnu_pcoa.tsv", quote=FALSE, sep="\t" )

# calculate variance
otu.pcoa=cmdscale(otu.dist, eig = TRUE)
otu.pcoa$eig[1]/sum(otu.pcoa$eig)
otu.pcoa$eig[2]/sum(otu.pcoa$eig)

# upload mapping data (metadata)
map=read_delim("burn_pcoa_metadata.txt", delim = "\t", col_names = TRUE)
ggplot(map, aes(x=pcoa1, y=pcoa2, color=Burned, shape=Plot))+
  geom_point()
    
# PERMANOVA
adonis(otu.dist
~map$'variableofinterest')

# p-value - probability that the result would happen by random chance, significance 
# R^2 - how close of an association 
# pseudo F - slope of line

# if multiple variables are significant, the one with higher F. model may be contributing more to the differences